# Quantum Reservoir Computing (Fujii-Nakajima) による時系列2値分類

このNotebookは、**Google Colab** でそのまま実行できるQRCデモです。

- リポジトリをColab上でclone
- 可視化の詳細コードは `src/spin_viz.py` をimportして利用
- QRC本体（入力注入・時間発展・リセット・分類）はNotebook内で明示


## 0. Colabセットアップ

`REPO_URL` を必要に応じて変更して実行してください。


In [ ]:
import os
import sys
import pathlib
import subprocess

# 実行時の .pyc 生成を抑制（リポジトリ汚染を減らす）
os.environ["PYTHONDONTWRITEBYTECODE"] = "1"

REPO_URL = "https://github.com/dainfinity/qrc_classification_demo.git"
REPO_DIR = pathlib.Path("/content/qrc_classification_demo")

if not REPO_DIR.exists():
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)

subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", str(REPO_DIR / "requirements-colab.txt")], check=True)

if str(REPO_DIR) not in sys.path:
    sys.path.insert(0, str(REPO_DIR))

DATA_DIR = REPO_DIR / "data"
OUTPUT_DIR = pathlib.Path("/content/qrc_outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Repo:", REPO_DIR)
print("Data:", DATA_DIR)
print("Output:", OUTPUT_DIR)


## 1. データ読み込みとタスク説明

このデモの2値分類タスクは、**同じ周波数成分を含むが時間順序だけが異なる系列**の識別です。

- `class 0`: 前半は低周波、後半は高周波（low -> high）
- `class 1`: 前半は高周波、後半は低周波（high -> low）
- 両クラスは使う周波数成分自体は同じで、順序だけが異なるため、時間順序の記憶が必要です。
- 低周波/高周波のレンジは少し重なるように設定しています。
- 切り替え時刻は中央付近に固定し、ノイズ量を大きめ（`noise_std=0.4`）にしています。

CSVファイル:
- `data/train_X.csv`, `data/train_y.csv`
- `data/test_X.csv`, `data/test_y.csv`

各系列の長さは `L=96`、サンプル数は train/test それぞれ100本です。


In [ ]:
import os
from itertools import repeat
from concurrent.futures import ProcessPoolExecutor

import numpy as np
import pandas as pd
from scipy.linalg import expm
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import HTML, display
from tqdm import tqdm

SEED = 1234
np.random.seed(SEED)

USE_PARALLEL = True
N_WORKERS = min(4, os.cpu_count() or 2)
print(f"USE_PARALLEL={USE_PARALLEL}, N_WORKERS={N_WORKERS}, SEED={SEED}")

train_X_raw = pd.read_csv(DATA_DIR / "train_X.csv").values.astype(np.float64)
train_y = pd.read_csv(DATA_DIR / "train_y.csv")["label"].values.astype(int)
test_X_raw = pd.read_csv(DATA_DIR / "test_X.csv").values.astype(np.float64)
test_y = pd.read_csv(DATA_DIR / "test_y.csv")["label"].values.astype(int)

M_train, L = train_X_raw.shape
M_test, L2 = test_X_raw.shape
assert L == L2
print(f"Train: {train_X_raw.shape}, Test: {test_X_raw.shape}, sequence length L={L}")

n0_train = int((train_y == 0).sum())
n1_train = int((train_y == 1).sum())
n0_test = int((test_y == 0).sum())
n1_test = int((test_y == 1).sum())
print(f"Train labels: class0={n0_train}, class1={n1_train}")
print(f"Test labels : class0={n0_test}, class1={n1_test}")

# データの雰囲気を確認（各クラスから3本ずつ）
rng_preview = np.random.default_rng(SEED)
idx0 = np.where(train_y == 0)[0]
idx1 = np.where(train_y == 1)[0]
show0 = rng_preview.choice(idx0, size=min(3, len(idx0)), replace=False)
show1 = rng_preview.choice(idx1, size=min(3, len(idx1)), replace=False)

fig, axes = plt.subplots(1, 2, figsize=(12, 3.6), sharey=True)
for i in show0:
    axes[0].plot(train_X_raw[i], alpha=0.9)
for i in show1:
    axes[1].plot(train_X_raw[i], alpha=0.9)
axes[0].set_title("class 0: low -> high")
axes[1].set_title("class 1: high -> low")
for ax in axes:
    ax.set_xlabel("time index")
    ax.grid(alpha=0.25)
axes[0].set_ylabel("signal value")
plt.tight_layout()
plt.show()


## 2. 多体系スピン可視化とハイパーパラメータ

`src/spin_viz.py` のウィジェットで以下を操作します。

- トポロジー: `all_to_all`, `chain_1d`
- スピン数: `N`
- 結合幅: `J width`（**0.00 ~ 2.00**）
- 横磁場: `h field`（**0.00 ~ 2.00**）
- 時間発展長: `tau`（**0.01 ~ 50.00**）

Fujii-Nakajimaスキームで調整する主要パラメータは、`J width`, `h field`, `tau` です。


In [ ]:
from src.spin_viz import build_spin_control_panel

panel, controls = build_spin_control_panel(default_n=4, seed=SEED)
display(panel)

tau_slider = widgets.FloatSlider(value=1.00, min=0.01, max=50.00, step=0.01, description="tau")
display(tau_slider)

print("計算前に上のパラメータを調整してください。")



## 3. Fujii-Nakajima スキーム（入力注入 → 時間発展 → リセット）

### 3.1 入力の正規化（必須）
各時刻の入力を必ず `[0,1]` にスケーリングしてから注入します:

$$
\tilde{u}_t = \mathrm{clip}\left(\frac{u_t-u_{\min}}{u_{\max}-u_{\min}},\,0,\,1\right)
$$

### 3.2 Ry回転による入力注入
入力サイト（ここでは0番サイト）に `Ry` 回転で入力を注入します:

$$
\theta_t = \frac{\pi}{2}\tilde{u}_t, \qquad
|\psi_{\mathrm{in}}(\tilde{u}_t)\rangle = R_y(\theta_t)|0\rangle
$$

$$
R_y(\theta)=
\begin{pmatrix}
\cos(\theta/2) & -\sin(\theta/2)\\
\sin(\theta/2) &  \cos(\theta/2)
\end{pmatrix}
$$

密度行列は

$$
\rho_t^{\mathrm{in}} = |\psi_{\mathrm{in}}\rangle\langle\psi_{\mathrm{in}}|
$$

### 3.3 多体系ハミルトニアンで時間発展

$$
H = \sum_{(i,j)\in E} J_{ij} Z_i Z_j + h\sum_{i=0}^{N-1}X_i,
\qquad
U = e^{-iH\tau}
$$

- $E$: 選択したトポロジーで決まるエッジ集合
- $J_{ij}\sim\mathrm{Uniform}[-J_{\mathrm{width}}, J_{\mathrm{width}}]$

時刻 $t$ では、リザバー内部状態 $\rho_{t-1}^{\mathrm{res}}$ と入力を合成して

$$
\rho_t^{\mathrm{joint}} = \rho_t^{\mathrm{in}} \otimes \rho_{t-1}^{\mathrm{res}},
\qquad
\rho_t' = U\rho_t^{\mathrm{joint}}U^\dagger
$$

### 3.4 観測とリセット
観測量は各サイトの $Z$ 期待値:

$$
r_t^{(i)} = \mathrm{Tr}(\rho_t' Z_i),\quad i=0,\dots,N-1
$$

次時刻へ渡す内部状態は、入力サイトを部分トレースして

$$
\rho_t^{\mathrm{res}} = \mathrm{Tr}_{\mathrm{in}}(\rho_t')
$$

とします。これが「入力注入-時間発展-リセット」の核です。


In [ ]:
SIGMA_X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
SIGMA_Z = np.array([[1, 0], [0, -1]], dtype=np.complex128)
IDENTITY = np.eye(2, dtype=np.complex128)


def kron_many(ops):
    out = np.array([[1.0 + 0.0j]])
    for op in ops:
        out = np.kron(out, op)
    return out


def site_operator(n_spins, site, op):
    mats = [IDENTITY] * n_spins
    mats[site] = op
    return kron_many(mats)


def zz_operator(n_spins, i, j):
    mats = [IDENTITY] * n_spins
    mats[i] = SIGMA_Z
    mats[j] = SIGMA_Z
    return kron_many(mats)


def build_edges(n_spins, topology):
    if topology == "all_to_all":
        return [(i, j) for i in range(n_spins) for j in range(i + 1, n_spins)]
    if topology == "chain_1d":
        return [(i, i + 1) for i in range(n_spins - 1)]
    raise ValueError(f"Unknown topology: {topology}. Use 'all_to_all' or 'chain_1d'.")


def build_hamiltonian_and_observables(n_spins, topology, j_width, h_field, seed=SEED):
    x_ops = [site_operator(n_spins, i, SIGMA_X) for i in range(n_spins)]
    z_ops = [site_operator(n_spins, i, SIGMA_Z) for i in range(n_spins)]
    edges = build_edges(n_spins, topology)

    rng = np.random.default_rng(seed)
    dim = 2 ** n_spins
    h_int = np.zeros((dim, dim), dtype=np.complex128)
    for (i, j) in edges:
        jij = rng.uniform(-j_width, j_width)
        h_int += jij * zz_operator(n_spins, i, j)

    h_field_term = np.zeros((dim, dim), dtype=np.complex128)
    for i in range(n_spins):
        h_field_term += h_field * x_ops[i]

    h0 = h_int + h_field_term
    return h0, z_ops


def minmax_scale_to_unit_interval(x, u_min, u_max):
    if abs(u_max - u_min) < 1e-12:
        return np.zeros_like(x)
    z = (x - u_min) / (u_max - u_min)
    return np.clip(z, 0.0, 1.0)


def ry_matrix(theta):
    c = np.cos(theta / 2.0)
    s = np.sin(theta / 2.0)
    return np.array([[c, -s], [s, c]], dtype=np.complex128)


def pure_input_density_from_u(u01):
    u01 = float(np.clip(u01, 0.0, 1.0))
    theta = 0.5 * np.pi * u01
    ket0 = np.array([1.0, 0.0], dtype=np.complex128)
    psi = ry_matrix(theta) @ ket0
    return np.outer(psi, psi.conj())


def partial_trace_input_qubit(rho_joint, n_spins):
    d_res = 2 ** (n_spins - 1)
    rho4 = rho_joint.reshape(2, d_res, 2, d_res)
    return np.trace(rho4, axis1=0, axis2=2)


def sequence_to_state_matrix_fn(sequence_raw, params):
    n_spins = params["n_spins"]
    topology = params["topology"]
    j_width = params["j_width"]
    h_field = params["h_field"]
    tau = params["tau"]
    seed = params["seed"]
    u_min = params["u_min"]
    u_max = params["u_max"]

    h0, z_ops = build_hamiltonian_and_observables(n_spins, topology, j_width, h_field, seed=seed)
    u_op = expm(-1j * h0 * tau)
    u_dag = u_op.conj().T

    d_res = 2 ** (n_spins - 1)
    rho_res = np.zeros((d_res, d_res), dtype=np.complex128)
    rho_res[0, 0] = 1.0 + 0.0j

    sequence = minmax_scale_to_unit_interval(np.asarray(sequence_raw, dtype=np.float64), u_min, u_max)
    states = np.zeros((len(sequence), n_spins), dtype=np.float64)

    for t, u_t in enumerate(sequence):
        rho_in = pure_input_density_from_u(u_t)
        rho_joint = np.kron(rho_in, rho_res)
        rho_evolved = u_op @ rho_joint @ u_dag

        for i in range(n_spins):
            states[t, i] = float(np.real(np.trace(rho_evolved @ z_ops[i])))

        rho_res = partial_trace_input_qubit(rho_evolved, n_spins)

    return states


def build_matrices_for_dataset(X_raw, params, use_parallel=True, n_workers=2, desc="dataset"):
    total = len(X_raw)
    if use_parallel:
        try:
            with ProcessPoolExecutor(max_workers=n_workers) as ex:
                iterator = ex.map(sequence_to_state_matrix_fn, X_raw, repeat(params))
                mats = list(tqdm(iterator, total=total, desc=desc, leave=False, dynamic_ncols=True))
            return np.stack(mats, axis=0)
        except Exception as e:
            print(f"Parallel execution failed ({e}), fallback to sequential.")

    mats = []
    for seq in tqdm(X_raw, total=total, desc=desc, leave=False, dynamic_ncols=True):
        mats.append(sequence_to_state_matrix_fn(seq, params))
    return np.stack(mats, axis=0)


def predict_1nn_frobenius(train_mats, train_labels, test_mats, show_progress=True):
    preds = []
    iterator = test_mats
    if show_progress:
        iterator = tqdm(test_mats, total=len(test_mats), desc="1-NN prediction", leave=False, dynamic_ncols=True)

    for test_mat in iterator:
        diffs = train_mats - test_mat[None, :, :]
        dists = np.linalg.norm(diffs, ord="fro", axis=(1, 2))
        nn_idx = int(np.argmin(dists))
        preds.append(int(train_labels[nn_idx]))
    return np.array(preds, dtype=int)


def show_reservoir_params(params):
    topology_label = {
        "all_to_all": "All-to-all",
        "chain_1d": "1D Chain",
    }.get(params["topology"], params["topology"])

    html = f"""
    <div style='font-family: ui-sans-serif, -apple-system, Segoe UI, Roboto, sans-serif; margin: 8px 0 14px;'>
      <div style='display:flex; gap:10px; flex-wrap:wrap; align-items:flex-start;'>
        <div style='background:#0f172a;color:#e2e8f0;padding:10px 12px;border-radius:10px;min-width:170px;box-shadow:0 2px 8px rgba(0,0,0,.12);'>
          <div style='font-size:11px;opacity:.75;'>Topology</div>
          <div style='font-size:16px;font-weight:700;color:#93c5fd;'>{topology_label}</div>
        </div>
        <div style='background:#082f49;color:#e0f2fe;padding:10px 12px;border-radius:10px;min-width:130px;'>
          <div style='font-size:11px;opacity:.75;'>N spins</div>
          <div style='font-size:16px;font-weight:700;'>{params['n_spins']}</div>
        </div>
        <div style='background:#3f1d2e;color:#fce7f3;padding:10px 12px;border-radius:10px;min-width:130px;'>
          <div style='font-size:11px;opacity:.75;'>J width</div>
          <div style='font-size:16px;font-weight:700;'>{params['j_width']:.3f}</div>
        </div>
        <div style='background:#422006;color:#ffedd5;padding:10px 12px;border-radius:10px;min-width:130px;'>
          <div style='font-size:11px;opacity:.75;'>h field</div>
          <div style='font-size:16px;font-weight:700;'>{params['h_field']:.3f}</div>
        </div>
        <div style='background:#052e16;color:#dcfce7;padding:10px 12px;border-radius:10px;min-width:130px;'>
          <div style='font-size:11px;opacity:.75;'>tau</div>
          <div style='font-size:16px;font-weight:700;'>{params['tau']:.3f}</div>
        </div>
      </div>
      <div style='margin-top:8px;font-size:12px;color:#475569;'>
        input scale range: [{params['u_min']:.4f}, {params['u_max']:.4f}] &nbsp; | &nbsp; seed: {params['seed']}
      </div>
      </div>
    </div>
    <script>
      if (window.MathJax && window.MathJax.typesetPromise) {
        window.MathJax.typesetPromise();
      }
    </script>
    """
    display(HTML(html))


## 4. 実行（スケーリング → リザバー行列化 → 1-NN）

- 入力は必ず `[0,1]` へ正規化
- 各サンプルを `L × N` のリザバー状態行列へ変換
- テスト系列は、訓練行列とのフロベニウス距離で1-NN分類


In [ ]:
# 入力スケーリング統計量（train側）
u_min = float(train_X_raw.min())
u_max = float(train_X_raw.max())
print(f"Input scaling range from train: u_min={u_min:.4f}, u_max={u_max:.4f}")

reservoir_params = {
    "n_spins": int(controls["n_spins"].value),
    "topology": controls["topology"].value,
    "j_width": float(controls["j_width"].value),
    "h_field": float(controls["h_field"].value),
    "tau": float(tau_slider.value),
    "seed": SEED,
    "u_min": u_min,
    "u_max": u_max,
}

show_reservoir_params(reservoir_params)

train_mats = build_matrices_for_dataset(
    train_X_raw, reservoir_params, use_parallel=USE_PARALLEL, n_workers=N_WORKERS, desc="Train reservoir states"
)
test_mats = build_matrices_for_dataset(
    test_X_raw, reservoir_params, use_parallel=USE_PARALLEL, n_workers=N_WORKERS, desc="Test reservoir states"
)

# 保存はデフォルトOFF（Colab上で不要な変更扱いを減らす）
SAVE_ARTIFACTS = False
if SAVE_ARTIFACTS:
    np.save(OUTPUT_DIR / "train_reservoir_matrices.npy", train_mats)
    np.save(OUTPUT_DIR / "test_reservoir_matrices.npy", test_mats)
    print("saved:", OUTPUT_DIR / "train_reservoir_matrices.npy")
    print("saved:", OUTPUT_DIR / "test_reservoir_matrices.npy")

pred = predict_1nn_frobenius(train_mats, train_y, test_mats, show_progress=True)
acc = (pred == test_y).mean()
acc_color = "#065f46" if acc >= 0.9 else ("#92400e" if acc >= 0.75 else "#991b1b")
display(HTML(
    f"<div style='margin-top:10px;padding:10px 12px;border-radius:10px;background:#f8fafc;border:1px solid #e2e8f0;"
    "font-family:ui-sans-serif, -apple-system, Segoe UI, Roboto;'>"
    "<span style='font-size:12px;color:#64748b;'>1-NN accuracy (Frobenius)</span>"
    f"<div style='font-size:22px;font-weight:800;color:{acc_color};line-height:1.2;'>{acc:.3f}</div>"
    "</div>"
))


## 5. 実装上の注意

- 本Notebookは **時間多重化なし**
- 特徴は各時刻で得る `N` 次元ベクトルのみ
- サンプル間は独立なので、可能な範囲で並列処理を使う
